In [188]:
import pandas as pd
import numpy as np

In [169]:
data_list = []
f = open("Data/station_description.txt")
for line in f:
    lst = []
    contents = " ".join(line.split())
    contents = contents.split(" ")
    lst.append(contents[0]) # id
    lst.append(contents[1]) # from_date
    lst.append(contents[2]) # to_date
    lst.append(contents[3]) # height
    lst.append(contents[4]) # geoBreite
    lst.append(contents[5]) # geoLaenge
    lst.append(" ".join(contents[6:-1])) # name
    lst.append(contents[-1]) # area
    data_list.append(lst)
f.close()
columns = data_list.pop(0)
_ = data_list.pop(0)

In [170]:
for line in data_list:
    if len(line)>8:
        print(line)

In [171]:
columns

['Stations_id',
 'von_datum',
 'bis_datum',
 'Stationshoehe',
 'geoBreite',
 'geoLaenge',
 'Stationsname',
 'Bundesland']

In [174]:
df= pd.DataFrame(data_list, columns=columns).set_index("Stations_id")
df = df.astype({"von_datum": "int32", "bis_datum": "int32", "Stationshoehe": "int32", "geoBreite": "float64", "geoLaenge": "float64"})

In [175]:
df.head()

,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland
Stations_id,,,,,,,
00001,19370101,19860630,478,47.8413,8.8493,Aach,Baden-Württemberg
00003,18910101,20110331,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
00011,19800901,20200930,680,47.9737,8.5205,Donaueschingen (Landeplatz),Baden-Württemberg
00044,19690101,20210921,44,52.9336,8.2370,Großenkneten,Niedersachsen
00052,19690101,20011231,46,53.6623,10.1990,Ahrensburg-Wulfsdorf,Schleswig-Holstein


In [176]:
current_stations = df[df["bis_datum"]>20210000]
current_stations.head()

,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland
Stations_id,,,,,,,
00044,19690101,20210921,44,52.9336,8.2370,Großenkneten,Niedersachsen
00073,19590301,20210921,340,48.6159,13.0506,Aldersbach-Kriestorf,Bayern
00078,19610101,20210921,65,52.4853,7.9126,Alfhausen,Niedersachsen
00090,19880219,20210921,305,50.7557,9.2583,Alsfeld,Hessen
00091,19781101,20210921,300,50.7446,9.3450,Alsfeld-Eifa,Hessen


In [182]:
relevant_laenge = current_stations[(current_stations["geoLaenge"]>7)&(current_stations["geoLaenge"]<9)]
relevant = relevant_laenge[(relevant_laenge["geoBreite"] < 51)& (relevant_laenge["geoBreite"] >49)]
relevant.head()

,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland
Stations_id,,,,,,,
00150,19510101,20210921,215,49.7273,8.1164,Alzey,Rheinland-Pfalz
00161,19801201,20210921,75,50.4237,7.4202,Andernach,Rheinland-Pfalz
00330,19370101,20210921,455,49.5617,8.9673,Oberzent-Beerfelden,Hessen
00377,19470101,20210921,210,49.1070,7.9967,"Bergzabern, Bad",Rheinland-Pfalz
00390,19861201,20210921,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen


In [186]:
filtered = relevant[relevant["Bundesland"].isin(["Rheinland-Pfalz", "Nordrhein-Westfalen", "Saarland"])]

In [192]:
breite_erftstadt = 50.7948
laenge_erftstadt = 6.7775
filtered["distance_erftstadt"] = np.sqrt((filtered["geoBreite"] - breite_erftstadt)*(filtered["geoBreite"] - breite_erftstadt) + (filtered["geoLaenge"] - laenge_erftstadt)*(filtered["geoLaenge"] - laenge_erftstadt))
filtered.head()

C:\Users\NHG\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland,distance_erftstadt
Stations_id,,,,,,,,
00150,19510101,20210921,215,49.7273,8.1164,Alzey,Rheinland-Pfalz,1.712370
00161,19801201,20210921,75,50.4237,7.4202,Andernach,Rheinland-Pfalz,0.742145
00377,19470101,20210921,210,49.1070,7.9967,"Bergzabern, Bad",Rheinland-Pfalz,2.082094
00390,19861201,20210921,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen,1.601976
00535,19470101,20210921,417,50.0372,7.3079,Blankenrath,Rheinland-Pfalz,0.924815


In [193]:
breite_schuld = 50.4467
laenge_schuld = 6.88944
filtered["distance_schuld"] = np.sqrt((filtered["geoBreite"] - breite_schuld)*(filtered["geoBreite"] - breite_schuld) + (filtered["geoLaenge"] - laenge_schuld)*(filtered["geoLaenge"] - laenge_schuld))
filtered.head()

C:\Users\NHG\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland,distance_erftstadt,distance_schuld
Stations_id,,,,,,,,,
00150,19510101,20210921,215,49.7273,8.1164,Alzey,Rheinland-Pfalz,1.712370,1.422311
00161,19801201,20210921,75,50.4237,7.4202,Andernach,Rheinland-Pfalz,0.742145,0.531258
00377,19470101,20210921,210,49.1070,7.9967,"Bergzabern, Bad",Rheinland-Pfalz,2.082094,1.738051
00390,19861201,20210921,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen,1.601976,1.573339
00535,19470101,20210921,417,50.0372,7.3079,Blankenrath,Rheinland-Pfalz,0.924815,0.585490


In [196]:
filtered["mean_distance"] = (filtered["distance_erftstadt"] + filtered["distance_schuld"])/2
filtered.sort_values("mean_distance")

C:\Users\NHG\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland,distance_erftstadt,distance_schuld,mean_distance
Stations_id,,,,,,,,,,
03490,19480101,20210714,111,50.5346,7.0853,"Neuenahr, Bad-Ahrweiler",Rheinland-Pfalz,0.403045,0.214680,0.308862
00603,19860801,20210921,147,50.7293,7.2040,Königswinter-Heiderhof,Nordrhein-Westfalen,0.431500,0.422860,0.427180
02667,19570701,20210921,92,50.8646,7.1575,Köln-Bonn,Nordrhein-Westfalen,0.386357,0.496484,0.441421
00766,19650101,20210921,477,50.1745,7.0595,Büchel (Flugplatz),Rheinland-Pfalz,0.681393,0.320957,0.501175
03540,19920901,20210921,195,50.8446,7.3720,Neunkirchen-Seelscheid-Krawinkel,Nordrhein-Westfalen,0.596582,0.625451,0.611016
15044,20111001,20210921,199,50.4517,7.4058,Neuwied-Feldkirchen-Hüllenberg,Rheinland-Pfalz,0.715876,0.516384,0.616130
00161,19801201,20210921,75,50.4237,7.4202,Andernach,Rheinland-Pfalz,0.742145,0.531258,0.636701
02362,19821001,20210921,328,50.5651,7.4843,Hümmerich,Rheinland-Pfalz,0.743188,0.606529,0.674858
06242,20181019,20210701,64,50.4088,7.4890,Mülheim-Kärlich,Rheinland-Pfalz,0.809462,0.600757,0.705109
